In [4]:
from torchvision.models.detection.rpn import RegionProposalNetwork, AnchorGenerator, RPNHead
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models.detection.roi_heads import roi_align
from torch.utils.data.dataloader import DataLoader
from models.backbone.ResNet import resnet12
from torchvision.models.detection.image_list import ImageList
from utils.dataset_tools.coco import read_coco_detection, show_dataset
from torchvision.transforms import transforms

# root : coco图片路径
# annFile : 标注文件路径
# transform : 图像转换(用于PIL)
# target_transform : 标注转换
# transforms : 图像和标注的转换

# 数据集预处理
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
normalize = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=mean, std=std)])
t = transforms.Compose([transforms.ToTensor(),
                        transforms.Resize(600),  # 短边缩为600
                        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
                        transforms.RandomHorizontalFlip(0.5)])

# 读取数据集
fsod_dataset = read_coco_detection(img_transform=t)

# support选取


mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
normalize = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize(mean=mean, std=std)])
transforms = transforms.Compose([transforms.ToTensor(),
                                 transforms.Resize((600, 600)),
                                 transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
                                 transforms.RandomHorizontalFlip(0.5)])
fsod_dataset = read_coco_detection(img_transform=transforms)

# for i in range(len(fsod_dataset)):
#     img, labels = fsod_dataset.__getitem__(i)
#     print(img.shape)
#     print(len(labels))

fsod_dataloader = DataLoader(fsod_dataset, batch_size=5, shuffle=True, drop_last=True)

for img, labels in enumerate(fsod_dataloader):
    print(img)

loading annotations into memory...
Done (t=1.43s)
creating index...
index created!
loading annotations into memory...
Done (t=0.97s)
creating index...
index created!


RuntimeError: each element in list of batch should be of equal size

In [ ]:
# N-way K-shot:
# support: N个类别, 每个类别K个实例
# query: 不限, 只要在N个类别内即可

# 构建N类support, 每个类别的support含有k个实例

def generate_support():
    pass

In [19]:
import json
from torch.utils.data import Dataset
from utils.dataset_tools.coco import read_coco_detection
from PIL import Image


def class_dict(cocoDataset: Dataset):
    class_dicts = {}
    for img, labels in enumerate(cocoDataset):
        print(img, labels)
        for label in labels:
            class_index = label['category_id']
            if not class_index in class_dicts.keys():
                class_dicts[class_index] = []
                class_dicts[class_index].append(img.filename)
            else:
                if not img.filename in class_dicts[class_index]:
                    class_dicts[class_index].append(img.filename)
    json.dump(class_dicts, fp='class_dicts')


mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
# normalize = transforms.Compose([transforms.ToTensor(),
#                                 transforms.Normalize(mean=mean, std=std)])
# transforms = transforms.Compose([transforms.ToTensor(),
#                                  transforms.Resize((600, 600)),
#                                  transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
#                                  transforms.RandomHorizontalFlip(0.5)])
fsod_dataset = read_coco_detection(img_transform=None)

# for i in range(len(fsod_dataset)):
#     img, labels = fsod_dataset.__getitem__(i)
#     print(img.shape)
#     print(len(labels))

class_dict(fsod_dataset)

img = Image.open('datasets/fsod/images/part_1/n00480993/n00480993_11037.jpg')
print(img, img.filename)

loading annotations into memory...
Done (t=1.86s)
creating index...
index created!
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=333x500 at 0x218889A5748> datasets/fsod/images/part_1/n00480993/n00480993_11037.jpg


In [ ]:
from utils.data.dataset import FsodDataset

fsod_dataset = FsodDataset(root='datasets/fsod', annFile='datasets/fsod/annotations/fsod_train.json', support_shot=2, query_shot=5)